In [1]:
# Python imports
import os
from datetime import datetime, timedelta
import pytz
import pandas as pd
import numpy as np     
import matplotlib.pyplot as plt
from dateutil import parser
%matplotlib inline
plt.style.use('seaborn-whitegrid')
pd.options.display.float_format = '{:,.2f}'.format
from IPython.display import display, HTML
import uuid
import time

# Lusid imports
import lusid
from lusid import models
from lusid.utilities import ApiClientFactory
from lusid.models import flow_conventions
from lusidjam import RefreshingToken 

# comment below for prod
#secrets_path = '/Users/nickmarriott/Library/CloudStorage/OneDrive-FinbourneTechnologyLimited/Work/01_2022_SOFR Swaps/Testing/secrets.json'#os.getenv("FBN_SECRETS_PATH")
secrets_path = os.getenv("FBN_SECRETS_PATH") 
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(), 
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)
complex_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
calendar_api = api_factory.build(lusid.api.CalendarsApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# Create required ccys if not present
instruments = {
            "1": models.InstrumentDefinition(name="GBP", identifiers={"Currency": models.InstrumentIdValue("GBP")}),
            "2": models.InstrumentDefinition(name="USD", identifiers={"Currency": models.InstrumentIdValue("USD")}),
            "3": models.InstrumentDefinition(name="EUR", identifiers={"Currency": models.InstrumentIdValue("EUR")}),
            "4": models.InstrumentDefinition(name="JPY", identifiers={"Currency": models.InstrumentIdValue("JPY")}),
            "5": models.InstrumentDefinition(name="ZZZ", identifiers={"Currency": models.InstrumentIdValue("ZZZ")}),
            }
    
instruments_api.upsert_instruments(instruments)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)


/opt/conda/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


LUSID Environment Initialised
LUSID SDK Version:  0.6.9351.0


In [2]:
#Scopes and dates
runid = str(int(round(time.time() * 0.01)))

marketSupplier = "Lusid"
recipeCode = "recipe_"+runid
scope = "scope_"+runid
portfolio_code = "BondPortfolio_"+runid

start_date = datetime(2022, 1, 4, tzinfo=pytz.utc)
maturity_date = datetime(2023, 1, 4, 0, tzinfo=pytz.utc)



#Holidays
uk_holidays = dict()
uk_holidays[datetime(2022, 5, 2, tzinfo=pytz.utc)] = "Early May bank holiday"
uk_holidays[datetime(2022, 4, 4, tzinfo=pytz.utc)] = "Easter Monday"
uk_holidays[datetime(2022, 4, 15, tzinfo=pytz.utc)] = "Good Friday"
uk_holidays[datetime(2022, 1, 3, tzinfo=pytz.utc)] = "New Year’s Day (substitute day)"
uk_holidays[datetime(2022, 1, 27, tzinfo=pytz.utc)] = "Random"
uk_holidays[datetime(2022, 6, 4, tzinfo=pytz.utc)] = "Spring bank holiday"
uk_holidays[datetime(2022, 6, 3, tzinfo=pytz.utc)] = "Platinum Jubilee bank holiday"
uk_holidays[datetime(2022, 8, 29, tzinfo=pytz.utc)] = "Summer bank holiday"
uk_holidays[datetime(2022, 12, 26, tzinfo=pytz.utc)] = "Boxing Day"
uk_holidays[datetime(2022, 12, 27, tzinfo=pytz.utc)] = "Christmas Day (substitute day)"

global_holidays = dict()
global_holidays[datetime(2022, 1, 1, tzinfo=pytz.utc)] = "Western New Year"
global_holidays[datetime(2022, 1, 6, tzinfo=pytz.utc)] = "Epiphany"
global_holidays[datetime(2022, 1, 7, tzinfo=pytz.utc)] = "Orthodox Christmas"
global_holidays[datetime(2022, 1, 14, tzinfo=pytz.utc)] = "Orthodox New Year"
global_holidays[datetime(2022, 1, 14, tzinfo=pytz.utc)] = "Tamil Thai Pongal Day"
global_holidays[datetime(2022, 1, 18, tzinfo=pytz.utc)] = "Thaipusam"
global_holidays[datetime(2022, 1, 20, tzinfo=pytz.utc)] = "Aquarius (UTC)"
global_holidays[datetime(2022, 2, 1, tzinfo=pytz.utc)] = "Chinese New Year / Spring Festival"
global_holidays[datetime(2022, 2, 1, tzinfo=pytz.utc)] = "Seol-nal / Korean New Year"
global_holidays[datetime(2022, 2, 1, tzinfo=pytz.utc)] = "Tet Nguyen Dan / Vietnamese New Year"
global_holidays[datetime(2022, 2, 2, tzinfo=pytz.utc)] = "Tsagaan Sar / Mongolian New Year (Year of the Water Tiger)"
global_holidays[datetime(2022, 2, 18, tzinfo=pytz.utc)] = "Pisces (UTC)"
global_holidays[datetime(2022, 2, 28, tzinfo=pytz.utc)] = "Lailat al Miraj / The Prophet's Ascension (Umm al-Qura)"
global_holidays[datetime(2022, 3, 1, tzinfo=pytz.utc)] = "Maha Shivaratree"
global_holidays[datetime(2022, 3, 1, tzinfo=pytz.utc)] = "Shrove Tuesday / Carnival"
global_holidays[datetime(2022, 3, 2, tzinfo=pytz.utc)] = "Ash Wednesday"
global_holidays[datetime(2022, 3, 3, tzinfo=pytz.utc)] = "Dangpa Losar / Bhutanese New Year (Year of the Male Water Tiger)"
global_holidays[datetime(2022, 3, 3, tzinfo=pytz.utc)] = "Losar / Tibetan New Year"
global_holidays[datetime(2022, 3, 7, tzinfo=pytz.utc)] = "Orthodox Clean Monday"
global_holidays[datetime(2022, 3, 8, tzinfo=pytz.utc)] = "International Women's Day"
global_holidays[datetime(2022, 3, 8, tzinfo=pytz.utc)] = "Orthodox Shrove Tuesday"
global_holidays[datetime(2022, 3, 9, tzinfo=pytz.utc)] = "Orthodox Ash Wednesday"
global_holidays[datetime(2022, 3, 17, tzinfo=pytz.utc)] = "Purim"
global_holidays[datetime(2022, 3, 18, tzinfo=pytz.utc)] = "Holi Phagwa"
global_holidays[datetime(2022, 3, 18, tzinfo=pytz.utc)] = "Lailat al Bara'a / Night of Emancipation (Umm al-Qura)"
global_holidays[datetime(2022, 3, 20, tzinfo=pytz.utc)] = "Aries (UTC)"
global_holidays[datetime(2022, 3, 20, tzinfo=pytz.utc)] = "Vernal Equinox / Spring (UTC)"
global_holidays[datetime(2022, 3, 23, tzinfo=pytz.utc)] = "Mi-Carême / Mid-Lent"
global_holidays[datetime(2022, 4, 2, tzinfo=pytz.utc)] = "Ougadi / Ugaadi"
global_holidays[datetime(2022, 4, 2, tzinfo=pytz.utc)] = "Start of Ramadan (Umm al-Qura)"
global_holidays[datetime(2022, 4, 7, tzinfo=pytz.utc)] = "Qing Ming Jie / Tomb Sweeping Day"
global_holidays[datetime(2022, 4, 10, tzinfo=pytz.utc)] = "Palm Sunday"
global_holidays[datetime(2022, 4, 10, tzinfo=pytz.utc)] = "Ram Navami"
global_holidays[datetime(2022, 4, 13, tzinfo=pytz.utc)] = "Laotian New Year"
global_holidays[datetime(2022, 4, 14, tzinfo=pytz.utc)] = "Cambodian New Year"
global_holidays[datetime(2022, 4, 14, tzinfo=pytz.utc)] = "Mahavir Jayanti / Janma Kalyanak"
global_holidays[datetime(2022, 4, 14, tzinfo=pytz.utc)] = "Maundy Thursday"
global_holidays[datetime(2022, 4, 15, tzinfo=pytz.utc)] = "Good Friday"
global_holidays[datetime(2022, 4, 16, tzinfo=pytz.utc)] = "Pesach / Jewish Passover"
global_holidays[datetime(2022, 4, 17, tzinfo=pytz.utc)] = "Burmese New Year"
global_holidays[datetime(2022, 4, 17, tzinfo=pytz.utc)] = "Easter"
global_holidays[datetime(2022, 4, 18, tzinfo=pytz.utc)] = "Easter Monday"
global_holidays[datetime(2022, 4, 18, tzinfo=pytz.utc)] = "Nuzul Quran / Revelation of the Qur'an (Umm al-Qura)"
global_holidays[datetime(2022, 4, 20, tzinfo=pytz.utc)] = "Taurus (UTC)"
global_holidays[datetime(2022, 4, 22, tzinfo=pytz.utc)] = "Orthodox Good Friday"
global_holidays[datetime(2022, 4, 23, tzinfo=pytz.utc)] = "Orthodox Holy Saturday"
global_holidays[datetime(2022, 4, 24, tzinfo=pytz.utc)] = "Orthodox Easter"
global_holidays[datetime(2022, 4, 25, tzinfo=pytz.utc)] = "Orthodox Easter Monday"
global_holidays[datetime(2022, 4, 28, tzinfo=pytz.utc)] = "Lailat al Qadr / Night of Destiny (Umm al-Qura)"
global_holidays[datetime(2022, 4, 29, tzinfo=pytz.utc)] = "Jumatul Bidah / Friday of Farewell (Umm al-Qura)"
global_holidays[datetime(2022, 5, 1, tzinfo=pytz.utc)] = "International Labour Day"
global_holidays[datetime(2022, 5, 2, tzinfo=pytz.utc)] = "Eid al Fitr / End of Ramadan (Umm al-Qura)"
global_holidays[datetime(2022, 5, 13, tzinfo=pytz.utc)] = "Great Prayer Day"
global_holidays[datetime(2022, 5, 21, tzinfo=pytz.utc)] = "Gemini (UTC)"
global_holidays[datetime(2022, 5, 26, tzinfo=pytz.utc)] = "Ascension Day"
global_holidays[datetime(2022, 6, 3, tzinfo=pytz.utc)] = "Duan Wu Jie / Dragon Boat Festival"
global_holidays[datetime(2022, 6, 5, tzinfo=pytz.utc)] = "Shavuot"
global_holidays[datetime(2022, 6, 5, tzinfo=pytz.utc)] = "Whit Sunday / Pentecost"
global_holidays[datetime(2022, 6, 6, tzinfo=pytz.utc)] = "Whit Monday"
global_holidays[datetime(2022, 6, 13, tzinfo=pytz.utc)] = "Orthodox Pentecost Monday"
global_holidays[datetime(2022, 6, 16, tzinfo=pytz.utc)] = "Corpus Christi"
global_holidays[datetime(2022, 6, 21, tzinfo=pytz.utc)] = "Cancer (UTC)"
global_holidays[datetime(2022, 6, 21, tzinfo=pytz.utc)] = "Summer Solstice / Summer (UTC)"
global_holidays[datetime(2022, 7, 8, tzinfo=pytz.utc)] = "Arafat Day (Umm al-Qura)"
global_holidays[datetime(2022, 7, 9, tzinfo=pytz.utc)] = "Eid al Adha / Feast of Sacrifice (Umm al-Qura)"
global_holidays[datetime(2022, 7, 22, tzinfo=pytz.utc)] = "Leo (UTC)"
global_holidays[datetime(2022, 7, 30, tzinfo=pytz.utc)] = "Islamic New Year / Hijra New Year (1444) (Umm al-Qura)"
global_holidays[datetime(2022, 8, 8, tzinfo=pytz.utc)] = "Ashura (Umm al-Qura)"
global_holidays[datetime(2022, 8, 15, tzinfo=pytz.utc)] = "Assumption"
global_holidays[datetime(2022, 8, 16, tzinfo=pytz.utc)] = "Parsi New Year"
global_holidays[datetime(2022, 8, 23, tzinfo=pytz.utc)] = "Virgo (UTC)"
global_holidays[datetime(2022, 8, 31, tzinfo=pytz.utc)] = "Ganesh Chaturthi / Vinayaka Chaturthi / Chaturthi Paksha"
global_holidays[datetime(2022, 9, 8, tzinfo=pytz.utc)] = "Onam / Thiruonam"
global_holidays[datetime(2022, 9, 10, tzinfo=pytz.utc)] = "Zhong Qu Jie / Mid Autumn Festival"
global_holidays[datetime(2022, 9, 14, tzinfo=pytz.utc)] = "Magal de Touba (Umm al-Qura)"
global_holidays[datetime(2022, 9, 23, tzinfo=pytz.utc)] = "Autumnal Equinox / Fall (UTC)"
global_holidays[datetime(2022, 9, 23, tzinfo=pytz.utc)] = "Libra (UTC)"
global_holidays[datetime(2022, 9, 26, tzinfo=pytz.utc)] = "Rosh Hashanah / Jewish New Year (5783)"
global_holidays[datetime(2022, 10, 4, tzinfo=pytz.utc)] = "Dassain"
global_holidays[datetime(2022, 10, 5, tzinfo=pytz.utc)] = "Yom Kippur"
global_holidays[datetime(2022, 10, 8, tzinfo=pytz.utc)] = "Mawlid an Nabi / The Prophet's Birthday (Umm al-Qura)"
global_holidays[datetime(2022, 10, 10, tzinfo=pytz.utc)] = "Canadian Thanksgiving"
global_holidays[datetime(2022, 10, 10, tzinfo=pytz.utc)] = "Sukkot"
global_holidays[datetime(2022, 10, 15, tzinfo=pytz.utc)] = "Baptism of the Prophet (Umm al-Qura)"
global_holidays[datetime(2022, 10, 23, tzinfo=pytz.utc)] = "Scorpio (UTC)"
global_holidays[datetime(2022, 10, 24, tzinfo=pytz.utc)] = "Northern Deepavali / Diwali Amavasya"
global_holidays[datetime(2022, 10, 24, tzinfo=pytz.utc)] = "Southern Deepavali / Diwali Krisna Chaturdasi"
global_holidays[datetime(2022, 10, 31, tzinfo=pytz.utc)] = "Reformation Day"
global_holidays[datetime(2022, 11, 4, tzinfo=pytz.utc)] = "All Saints' Day"
global_holidays[datetime(2022, 11, 2, tzinfo=pytz.utc)] = "All Souls' Day"
global_holidays[datetime(2022, 11, 8, tzinfo=pytz.utc)] = "Guru Nanak Jayanti"
global_holidays[datetime(2022, 11, 22, tzinfo=pytz.utc)] = "Sagittarius (UTC)"
global_holidays[datetime(2022, 11, 24, tzinfo=pytz.utc)] = "US Thanksgiving"
global_holidays[datetime(2022, 11, 27, tzinfo=pytz.utc)] = "First Sunday in Advent"
global_holidays[datetime(2022, 12, 4, tzinfo=pytz.utc)] = "Second Sunday in Advent"
global_holidays[datetime(2022, 12, 5, tzinfo=pytz.utc)] = "Immaculate Conception"
global_holidays[datetime(2022, 12, 11, tzinfo=pytz.utc)] = "Third Sunday in Advent"
global_holidays[datetime(2022, 12, 18, tzinfo=pytz.utc)] = "Fourth Sunday in Advent"
global_holidays[datetime(2022, 12, 21, tzinfo=pytz.utc)] = "Capricorn (UTC)"
global_holidays[datetime(2022, 12, 21, tzinfo=pytz.utc)] = "Winter Solstice / Winter (UTC)"
global_holidays[datetime(2022, 12, 25, tzinfo=pytz.utc)] = "Western Christmas"
global_holidays[datetime(2022, 12, 26, tzinfo=pytz.utc)] = "Boxing Day / Saint Stephen's Day"
global_holidays[datetime(2022, 12, 31, tzinfo=pytz.utc)] = "Western New Year's Eve"

In [3]:
# Set up the recipe and define default calendar scope
pricingContext = models.PricingContext(
    options=models.PricingOptions(
        model_selection=models.ModelSelection(
                library="Lusid",
                model="ConstantTimeValueOfMoney"
            ),
            window_valuation_on_instrument_start_end=False
    )
)

marketContext = models.MarketContext(
    market_rules=[],
    options=models.MarketOptions(
        default_supplier="Lusid",
        default_scope=scope,
        default_instrument_code_type="RIC",
        calendar_scope=scope # Set calendar scope
    )
)

recipeID = models.ConfigurationRecipe(
    scope=scope,
    code=recipeCode,
    pricing=pricingContext,
    market=marketContext,
    description="Test Recipe for Bond Calendar test"
)

# Upsert the recipe
response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(configuration_recipe=recipeID)
)

In [4]:
# create portfolio
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lusid.ApiException as e:
    if "PortfolioWithIdAlreadyExists" in (e.body):
        print(f"PortfolioWithIdAlreadyExists {portfolio_code}")

In [5]:
#create a new calendar
def CreateCalendar(calCode, hols, name, days):
    
    calendar_request = models.CreateCalendarRequest(
        calendar_id=models.ResourceId(
            scope=scope,
            code=calCode
        ),
        calendar_type='Holiday',
        weekend_mask=models.WeekendMask(
            days=days,
            time_zone='UTC'
        ),
        source_provider=name,
        properties = []

    )
    try:
        response = calendar_api.create_calendar(
            create_calendar_request=calendar_request
        )
    except lusid.ApiException as e:
        if "Unable to set alternate ID for this entity, as it conflicts with another value." in (e.body):
            print(f"Unable to set alternate ID for {calCode}, as it conflicts with another value.")


    for h in hols:

        # Add date to the calendar
        dateRequest = models.CreateDateRequest(
            date_id=name+"_"+h.strftime('%j'),
            from_utc=h,
            to_utc=h+timedelta(days=1),
            time_zone="UTC",
            description=hols[h].replace("’", "").replace("/",""),
            type="Holiday",
            source_data={},
            attributes=models.DateAttributes(
                irregular=False,
                irregular_session=False,
                new_hours=False,
                activity='None',
                first_open='None',
                last_open='None',
                first_close='None',
                last_close='None'
            )
        )


        response = calendar_api.add_date_to_calendar(
            scope=scope,
            code=calCode,
            create_date_request=dateRequest
        )
        


In [6]:
#Upsert calendars  
noHols = "NoHols"
ukHols = "UKHols"
globalHols = "GlobalHols"
 
CreateCalendar(noHols, [], "No-Holidays",[])
CreateCalendar(ukHols, uk_holidays, "UK-Holidays",["Saturday","Sunday"])
CreateCalendar(globalHols, global_holidays, "Gloabl-Holidays",["Saturday","Sunday","Monday","Tuesday","Wednesday"])

In [17]:
calendar_api.get_dates(
            scope=scope,
            code="UKHols"
        ).values

[{'attributes': {'activity': 'None',
                 'first_close': 'None',
                 'first_open': 'None',
                 'irregular': False,
                 'irregular_session': False,
                 'last_close': 'None',
                 'last_open': 'None',
                 'new_hours': False},
  'date_identifier': 'UK-Holidays_003',
  'description': 'New Years Day (substitute day)',
  'from_utc': datetime.datetime(2022, 1, 3, 0, 0, tzinfo=tzlocal()),
  'href': None,
  'local_date': '2022-01-03',
  'source_data': {},
  'timezone': 'UTC',
  'to_utc': datetime.datetime(2022, 1, 4, 0, 0, tzinfo=tzlocal()),
  'type': 'Holiday'},
 {'attributes': {'activity': 'None',
                 'first_close': 'None',
                 'first_open': 'None',
                 'irregular': False,
                 'irregular_session': False,
                 'last_close': 'None',
                 'last_open': 'None',
                 'new_hours': False},
  'date_identifier': 'UK-Holidays_027

In [7]:
# Create Bond
def CreateBond(bond_identifier, calendars, roll):
    
    currency = "GBP" #Currently this has to be the same as DomCcy
    payment_frequency = "1W"
    roll_convention = roll
    day_count_convention = "ActualActual"
    payment_calendars = calendars
    reset_calendars = calendars
    settle_days = 0
    reset_days = 0
    dom_ccy = "GBP"
    principal = 1
    coupon_rate = 0.03
    bond_name = "My Test Bond"


    # Flow convention using calendars
    flow_conventions = models.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention=roll_convention,
        day_count_convention=day_count_convention,
        payment_calendars=payment_calendars,
        reset_calendars=reset_calendars,
        settle_days=settle_days,
        reset_days=reset_days,
    )

    bond = models.Bond(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=dom_ccy,
        principal=principal,
        coupon_rate=coupon_rate,
        flow_conventions=flow_conventions,
        identifiers={},
        instrument_type="Bond",
        calculation_type="DayCountCoupon"
    )

    # Define the instrument to be upserted
    bond_definition = models.InstrumentDefinition(
        name=bond_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(bond_identifier)},
        definition=bond,
    )

    # Upsert the instrument
    upsert_request = {bond_identifier: bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    return upsert_response.values[bond_identifier].identifiers['LusidInstrumentId']


In [8]:
# Create the transaction
def CreateTransaction(luid):
    transaction_request = [
        models.TransactionRequest(
                transaction_id="txn_"+luid,
                type="StockIn",
                instrument_identifiers={
                    "Instrument/default/LusidInstrumentId": luid
                },
                transaction_date="2021-12-31T10:00:00Z",
                settlement_date="2021-12-31T10:00:00Z",
                units=100,
                transaction_price=models.TransactionPrice(price=100.76, type="Price"),
                total_consideration=models.CurrencyAndAmount(
                    amount=10076, currency="GBP"
                ),
            )
        ]

    response = transaction_portfolios_api.upsert_transactions(
        scope=scope, code=portfolio_code, transaction_request=transaction_request
    )
    
    print(f"Transaction txn_{luid} succesfully updated at time: {response.version.as_at_date}")
    
def CreateInstrumentAndTransaction(identifier, holidays, roll):
    luid = CreateBond(identifier, holidays, roll)
    CreateTransaction(luid)
    return luid

ukBond = "UkBond"
gloablBond = "gloablBond"
ukglobalBond = "ukGlobalBond"
basicBond = "basicBond"

ukBondLuid = CreateInstrumentAndTransaction(ukBond, [ukHols],"MF")
gloablBondLuid = CreateInstrumentAndTransaction(gloablBond, [globalHols],"MF")
ukglobalBondLuid = CreateInstrumentAndTransaction(ukglobalBond, [globalHols,ukHols],"MF")
basicBondLuid = CreateInstrumentAndTransaction(basicBond,[noHols],"NoAdjustment")

Transaction txn_LUID_00003D7X succesfully updated at time: 2022-05-31 16:30:25.411713+00:00
Transaction txn_LUID_00003D8T succesfully updated at time: 2022-05-31 16:30:26.054356+00:00
Transaction txn_LUID_00003D8U succesfully updated at time: 2022-05-31 16:30:27.643759+00:00
Transaction txn_LUID_00003D8V succesfully updated at time: 2022-05-31 16:30:28.204179+00:00


In [9]:
# Get cashflows to see how holidays affect payments
api_response = transaction_portfolios_api.get_portfolio_cash_flows(
    scope,
    portfolio_code,
    effective_at=start_date,
    window_start=start_date,
    window_end=maturity_date+timedelta(10),
    recipe_id_scope=scope,
    recipe_id_code=recipeCode)


In [10]:
# Print cashflows out side by side and see different payment dates
cashflows = dict()
cashflows['txn_'+ukBondLuid] = dict()
cashflows['txn_'+gloablBondLuid] = dict()
cashflows['txn_'+ukglobalBondLuid] = dict()
cashflows['txn_'+basicBondLuid] = dict()

for x in api_response.values:
    if str(x.payment_date) in cashflows[x.source_transaction_id]:
        cashflows[x.source_transaction_id][str(x.payment_date)]+=(x.amount)
    else:
        cashflows[x.source_transaction_id][str(x.payment_date)]=(x.amount)

ukbond = pd.DataFrame(cashflows['txn_'+ukBondLuid].items(), columns=['UK Date', 'UK Amount'])
globalbond = pd.DataFrame(cashflows['txn_'+gloablBondLuid].items(), columns=['Global Date', 'Gloabl Amount'])
ukglobalbond = pd.DataFrame(cashflows['txn_'+ukglobalBondLuid].items(), columns=['UKGlobal Date', 'UKGlobal Amount'])
basicbond = pd.DataFrame(cashflows['txn_'+basicBondLuid].items(), columns=['Basic Date', 'Basic Amount'])
pd.concat([ukbond, globalbond,ukglobalbond,basicbond], axis=1)

,UK Date,UK Amount,Global Date,Gloabl Amount,UKGlobal Date,UKGlobal Amount,Basic Date,Basic Amount
0,2022-01-05 00:00:00+00:00,0.01,2022-01-21 00:00:00+00:00,0.07,2022-01-21 00:00:00+00:00,0.07,2022-01-05 00:00:00+00:00,0.01
1,2022-01-12 00:00:00+00:00,0.06,2022-01-27 00:00:00+00:00,0.05,2022-01-28 00:00:00+00:00,0.06,2022-01-12 00:00:00+00:00,0.06
2,2022-01-19 00:00:00+00:00,0.06,2022-02-03 00:00:00+00:00,0.06,2022-02-03 00:00:00+00:00,0.05,2022-01-19 00:00:00+00:00,0.06
3,2022-01-26 00:00:00+00:00,0.06,2022-02-10 00:00:00+00:00,0.06,2022-02-10 00:00:00+00:00,0.06,2022-01-26 00:00:00+00:00,0.06
4,2022-02-02 00:00:00+00:00,0.06,2022-02-17 00:00:00+00:00,0.06,2022-02-17 00:00:00+00:00,0.06,2022-02-02 00:00:00+00:00,0.06
5,2022-02-09 00:00:00+00:00,0.06,2022-02-24 00:00:00+00:00,0.06,2022-02-24 00:00:00+00:00,0.06,2022-02-09 00:00:00+00:00,0.06
6,2022-02-16 00:00:00+00:00,0.06,2022-03-04 00:00:00+00:00,0.07,2022-03-04 00:00:00+00:00,0.07,2022-02-16 00:00:00+00:00,0.06
7,2022-02-23 00:00:00+00:00,0.06,2022-03-10 00:00:00+00:00,0.05,2022-03-10 00:00:00+00:00,0.05,2022-02-23 00:00:00+00:00,0.06
8,2022-03-02 00:00:00+00:00,0.06,2022-03-24 00:00:00+00:00,0.12,2022-03-24 00:00:00+00:00,0.12,2022-03-02 00:00:00+00:00,0.06
9,2022-03-09 00:00:00+00:00,0.06,2022-03-31 00:00:00+00:00,0.06,2022-03-31 00:00:00+00:00,0.06,2022-03-09 00:00:00+00:00,0.06
